Library

In [39]:
import pandas as pd
import re

Data Cleaning and Preprocessing

In [94]:
# take only report chat
df = pd.read_csv("assets/classifiedchat_context.csv")
cat_report = df[df['Category'] == 'report']

# lowercase all messages
report = cat_report['Message'].str.lower()

# remove unwanted characters
report = report.str.replace('_', '').str.replace('`', '').str.replace('.', '').str.replace(',', '').str.replace('*', '').str.lstrip().str.rstrip()

# remove all emojis
emojis = pd.read_csv('assets/emojis.csv')
emojis_list = emojis['emo'].tolist()

for emoji in emojis_list:
    report = report.str.replace(emoji, '')

#remove unwanted words
word_list = ['selesai', 'done']
for word in word_list:
    report = report.str.replace(word, '')


for i, message in enumerate(report):
    modified_text = re.sub(r'(?<=\D)(\d+)', r' \1', message)
    modified_text = re.sub(r'(?<=\d)([a-zA-Z])', r' \1', modified_text)
    report[i:i+1] = modified_text

# save to csv file
report.to_csv('report.csv', index=False)

In [96]:
book_list = pd.read_csv('assets/biblebooknames.csv')
book_list = book_list['Kitab'].tolist()

# Construct a regex pattern to match any of the names in the list along with numbers and '-'
pattern = r'(?:\b(?:' + '|'.join(book_list) + r')\b(?:\s*\d*-*\d*\s*)*)'

# Messages
message_list = report
processed_message = list()
for i in range(len(message_list)):
    text = message_list[i:i+1].values[0]
    # Find all matches for the pattern in the text
    matches = re.findall(pattern, text)

    # Construct the final output by joining the matches
    output = ' '.join(matches)
    processed_message.append(output)

dict_msg = {'processed': processed_message}
pd.DataFrame(dict_msg).to_csv('cleaned_messages.csv', index=False)

# Print the final output
# print("Output:", output)